In [1]:
from heter_graph import *
import pandas as pd

In [2]:
#!jt -tfs 10 -fs 10 -nfs 10 -dfs 10 -ofs 10 -t gruvboxd -cellw 1100

### Data

Target is Movie level predictions, as one of three classes (Action, Comedy, and Drama).<br> 
torch.unique(data['movie'].y) -> tensor([0, 1, 2])

In [2]:
data = IMDB('./')[0]

In [3]:
idx = torch.rand(4278)<0.8
train_idx = torch.where(idx)[0]
val_idx = torch.where(~idx)[0]

y = data['movie'].y
y_val = y.numpy()[val_idx]

In [5]:
data

HeteroData(
  movie={
    x=[4278, 3066],
    y=[4278],
    train_mask=[4278],
    val_mask=[4278],
    test_mask=[4278]
  },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={ edge_index=[2, 12828] },
  (director, to, movie)={ edge_index=[2, 4278] },
  (actor, to, movie)={ edge_index=[2, 12828] }
)

### Model-1, concat type features
Concat different types of node together and different types of edge together. Use types embedding to differentiate. Only possible when dimensions are the same across types<br>
Data(edge_index=[2, 34212], x=[11616, 3066], node_type=[11616], edge_type=[34212])<br>
torch.all(homo_data.x[:4278]==data['movie'].x) -> True

In [5]:
# hyper-parameters
layers = 8
in_d = 3066
d = 128
d_type = 256
out_d = 3
node_embed_types = 3
edge_embed_types = 4
heads = 4
dropout = 0.1

epochs = 10

In [6]:
model = GNN1(layers,d,d_type,in_d,out_d,node_embed_types,
                 edge_embed_types,heads,dropout,train_idx,val_idx,y,y_val).to('cuda')
out_data = get_data(model,data)
train_eval(model,out_data,epochs,y_val, val_idx, print_freq=1)

epoch:1, train:1.0527727603912354, F1-val:0.3664670658682635
epoch:2, train:0.9482905268669128, F1-val:0.4802395209580838
epoch:3, train:0.7721800208091736, F1-val:0.5676646706586826
epoch:4, train:0.590130090713501, F1-val:0.644311377245509
epoch:5, train:0.4362507462501526, F1-val:0.6491017964071857
epoch:6, train:0.3261515200138092, F1-val:0.6730538922155689
epoch:7, train:0.24249352514743805, F1-val:0.6706586826347305
epoch:8, train:0.1829950213432312, F1-val:0.6790419161676646
epoch:9, train:0.13785423338413239, F1-val:0.6766467065868264
micro:0.6826347305389222, macro:0.6806947519173838


{'micro': 0.6826347305389222, 'macro': 0.6806947519173838}

Hyper-parameter search

In [6]:
epochs = 15
fixed_para = {'in_d' : 3066,
                'out_d' : 3,
                'node_embed_types' : 3,
                'edge_embed_types' : 4,
                'train_idx':train_idx,'val_idx':val_idx,'y':y,'y_val':y_val}

In [7]:
def get_hyper1():
    return {'layers' : int(np.random.randint(2,12)),
            'd' : int(np.random.choice([64,128,256,512])),
            'd_type' : int(np.random.choice([64,128,256,512])),
            'heads' : int(np.random.choice([1,2,4,8])),
            'multiple_factor' : int(np.random.choice([1,2,4])),
            'dropout' : np.random.rand()/4,
}

In [8]:
perfs = []
for _ in range(40):
    hyper = get_hyper1()
    fixed_para.update(hyper)
    model = GNN1(**fixed_para).to('cuda')
    out_data = get_data(model,data)
    perf = train_eval(model,out_data,epochs,y_val, val_idx, print_freq=100)
    hyper.update(perf)
    perfs.append(hyper)
    print(hyper)

{'layers': 11, 'd': 256, 'd_type': 512, 'heads': 1, 'multiple_factor': 1, 'dropout': 0.12739007319305615, 'micro': 0.699748743718593, 'macro': 0.6977378769089927}
{'layers': 4, 'd': 512, 'd_type': 128, 'heads': 4, 'multiple_factor': 2, 'dropout': 0.11301703202670413, 'micro': 0.6620603015075377, 'macro': 0.6602096033125983}
{'layers': 9, 'd': 512, 'd_type': 64, 'heads': 4, 'multiple_factor': 2, 'dropout': 0.08766981343137653, 'micro': 0.664572864321608, 'macro': 0.6650591480098341}
{'layers': 3, 'd': 64, 'd_type': 256, 'heads': 1, 'multiple_factor': 1, 'dropout': 0.10320399660742904, 'micro': 0.5866834170854272, 'macro': 0.5830359640874242}
{'layers': 3, 'd': 256, 'd_type': 128, 'heads': 1, 'multiple_factor': 2, 'dropout': 0.034508754438563605, 'micro': 0.6758793969849246, 'macro': 0.6742838166957172}
{'layers': 6, 'd': 256, 'd_type': 64, 'heads': 8, 'multiple_factor': 2, 'dropout': 0.18973442044363134, 'micro': 0.6934673366834171, 'macro': 0.6914210209949436}
{'layers': 2, 'd': 512, '

In [13]:
perfs = pd.DataFrame(perfs)

In [16]:
perfs.sort_values('micro',ascending=False)

,layers,d,d_type,heads,multiple_factor,dropout,micro,macro
34,9,128,128,2,2,0.048005,0.701005,0.698804
0,11,256,512,1,1,0.127390,0.699749,0.697738
5,6,256,64,8,2,0.189734,0.693467,0.691421
31,6,256,128,4,2,0.081668,0.693467,0.692745
32,8,512,512,4,1,0.058180,0.693467,0.694385
29,8,256,256,4,4,0.100275,0.689698,0.687889
36,3,512,512,2,4,0.169705,0.684673,0.684782
10,3,128,64,2,2,0.231840,0.683417,0.680291
19,6,512,256,4,1,0.242793,0.683417,0.680182
18,6,512,512,4,4,0.232304,0.682161,0.680933


### Model-2
Separate msg passing for each edge type

In [4]:
# hyper-parameter
layers=3
d=256
in_d=3066
out_d=3
dropout=0.1
multiple_factor=4

epochs = 15

In [5]:
model = GNN2(layers,d,in_d,out_d,
                 dropout,train_idx,val_idx,y,y_val,multiple_factor).to('cuda')

In [6]:
out_data = get_data(model,data)

In [7]:
train_eval(model,out_data,epochs,y_val, val_idx, print_freq=1)

epoch:1, train:1.045916199684143, F1-val:0.37456242707117854
epoch:2, train:0.9516915082931519, F1-val:0.45507584597432904
epoch:3, train:0.4890880286693573, F1-val:0.5507584597432905
epoch:4, train:0.30252429842948914, F1-val:0.6137689614935823
epoch:5, train:0.18270739912986755, F1-val:0.661610268378063
epoch:6, train:0.1223299652338028, F1-val:0.6592765460910152
epoch:7, train:0.08753902465105057, F1-val:0.662777129521587
epoch:8, train:0.06362009793519974, F1-val:0.6557759626604434
epoch:9, train:0.051235299557447433, F1-val:0.6476079346557759
micro:0.661610268378063, macro:0.6602906087744249


{'micro': 0.661610268378063, 'macro': 0.6602906087744249}

Hyper-parameter search

In [4]:
epochs = 15
fixed_para = {'in_d' : 3066,
                'out_d' : 3,
                'train_idx':train_idx,'val_idx':val_idx,'y':y,'y_val':y_val}

In [5]:
def get_hyper2():
    return {'layers' : int(np.random.randint(2,12)),
            'd' : int(np.random.choice([64,128,256,512])),
            'multiple_factor' : int(np.random.choice([1,2,4])),
            'dropout' : np.random.rand()/4,
}

In [ ]:
perfs = []
for _ in range(40):
    hyper = get_hyper2()
    fixed_para.update(hyper)
    model = GNN2(**fixed_para).to('cuda')
    out_data = get_data(model,data)
    perf = train_eval(model,out_data,epochs,y_val, val_idx, print_freq=100)
    hyper.update(perf)
    perfs.append(hyper)
    print(hyper)

In [8]:
perfs = pd.DataFrame(perfs)

In [9]:
perfs.sort_values('micro',ascending=False)

,layers,d,multiple_factor,dropout,micro,macro
19,3,512,1,0.061489,0.695703,0.695326
15,6,64,1,0.216936,0.695703,0.695398
7,11,64,2,0.216942,0.695703,0.694390
10,4,256,2,0.246138,0.688734,0.686496
2,7,64,4,0.093073,0.686411,0.684353
16,5,128,1,0.155499,0.686411,0.685028
1,9,256,4,0.164887,0.686411,0.684777
20,2,128,2,0.152666,0.685250,0.684813
12,7,256,1,0.179726,0.685250,0.682861
11,8,64,1,0.072217,0.680604,0.680198
